# [75.06 / 95.58] Organización de Datos <br> Trabajo Práctico 2: Machine Learning
# Feature Selection

**Grupo 30: Datatouille**

**http://fdelmazo.github.io/7506-Datos/**

Fuentes:

* http://people.stat.sfu.ca/~lockhart/richard/350/08_2/lectures/VariableSelection/web.pdf

In [ ]:
def get_feature_selection():
    # Se esconde en un def para poder importarlo

    return {

        'best_features_progresivo': ['timestamp_last_checkout',
 'doy_last_checkout',
 'woy_last_checkout',
 'has_checkout_month_5',
 'total_checkouts_month_5',
 'total_session_checkouts_month_5',
 'dom_last_checkout',
 'days_between_last_event_and_checkout',
 'days_to_last_checkout',
 'dow_last_checkout',
 'total_events_months_1_to_4',
 'total_sessions_months_1_to_4',
 'total_viewed_products',
 'Samsung Galaxy Pocket 2 Duos',
 'Samsung Galaxy On 7',
 'Samsung Galaxy Note 8',
 'Samsung Galaxy Note Edge',
 'Samsung Galaxy S3 Mini',
 'Samsung Galaxy Note 5',
 'Samsung Galaxy Note 4',
 'Samsung Galaxy Note 3 Neo Duos',
 'Samsung Galaxy Note 3',
 'Samsung Galaxy Note 2 N7100',
 'Samsung Galaxy Mega Duos',
 'Samsung Galaxy J7 Prime',
 'Samsung Galaxy J7 PRO',
 'Samsung Galaxy S3 Duos',
 'Samsung Galaxy S4 Mini',
 'Samsung Galaxy S3 Neo Duos i9300i',
 'Samsung Galaxy S3 Slim Duos',
 'Samsung Galaxy S3 i9300',
 'Samsung Galaxy J7 2016 Metal',
 'Samsung Galaxy S4 Mini Duos',
 'Samsung Galaxy S4 i9500',
 'Samsung Galaxy S4 i9505',
 'Samsung Galaxy S4 i9515',
 'Samsung Galaxy S5',
 'Samsung Galaxy S5 Duos',
 'Samsung Galaxy S5 Mini',
 'Samsung Galaxy S5 Mini Duos',
 'Samsung Galaxy S5 New Edition',
 'Samsung Galaxy S5 New Edition Duos ',
 'Samsung Galaxy S6 Edge',
 'Samsung Galaxy J7 Neo',
 'Samsung Galaxy J7',
 'Samsung Galaxy S6 Flat',
 'Samsung Galaxy J5 Prime',
 'Samsung Galaxy A7 2016',
 'Samsung Galaxy A7',
 'Samsung Galaxy A5 2017',
 'Samsung Galaxy A5 2016',
 'Samsung Galaxy A5',
 'Samsung Galaxy A3 Duos',
 'Samsung Galaxy A3 2016',
 'Quantum YOU',
 'Quantum Muv Up',
 'Quantum Muv Pro',
 'Quantum Muv',
 'Quantum GO 4G',
 'Quantum GO 3G',
 'Motorola Moto Z2 Play',
 'Motorola Moto Z2 Force',
 'Motorola Moto Z Power Edition',
 'Motorola Moto Z Play',
 'Motorola Moto Z',
 'Motorola Moto X2',
 'Samsung Galaxy A7 2017',
 'Samsung Galaxy A9 Pro 2016',
 'Samsung Galaxy Core 2 Duos',
 'Samsung Galaxy J1 2016',
 'Samsung Galaxy J5 PRO',
 'Samsung Galaxy J5 2016 Metal',
 'Samsung Galaxy J5',
 'Samsung Galaxy J3',
 'Samsung Galaxy J2 Prime TV',
 'Samsung Galaxy J2 4G Duos TV',
 'Samsung Galaxy J2 4G Duos ',
 'Samsung Galaxy J1 Mini',
 'Samsung Galaxy Grand Duos i9082',
 'Samsung Galaxy Core Plus Duos TV',
 'Samsung Galaxy Gran Prime Duos TV',
 'Samsung Galaxy Gran Prime Duos',
 'Samsung Galaxy Gran Prime 3G Duos',
 'Samsung Galaxy Gran Neo Plus Duos',
 'Samsung Galaxy Gran Neo Duos',
 'Samsung Galaxy Gran 2 Duos TV',
 'Samsung Galaxy E7',
 'Samsung Galaxy E5 4G Duos',
 'Samsung Galaxy S6 Edge Plus',
 'Samsung Galaxy S8',
 'Samsung Galaxy S7',
 'iPad Mini 4 Wi-Fi + 4G'],
        
        'best_features_forward': ['doy_last_checkout',
 'total_events',
 'dom_last_event',
 'percentage_session_ad',
 'avg_events_per_session',
 'total_events_last_week'],
  
        'best_features_backward' :  ['total_checkouts', 'total_conversions', 'total_events', 
                                    'total_sessions', 'total_session_checkouts', 'total_session_conversions', 
                                    'avg_events_per_session', 'has_checkout', 'total_conversions_month_1', 
                                    'total_checkouts_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 
                                    'total_conversions_month_2', 'total_checkouts_month_2', 'has_checkout_month_2', 
                                    'has_conversion_month_2', 'total_conversions_month_3', 'total_checkouts_month_3',
                                    'has_checkout_month_3', 'has_conversion_month_3', 'has_checkout_month_4',
                                    'has_conversion_month_4', 'total_conversions_month_5', 'total_checkouts_month_5',
                                    'has_checkout_month_5', 'has_conversion_month_5', 'total_conversions_months_1_to_4',
                                    'total_checkouts_months_1_to_4', 'has_checkout_months_1_to_4', 'has_conversion_months_1_to_4', 
                                    'total_conversions_last_week', 'total_checkouts_last_week', 'has_checkout_last_week', 
                                    'has_conversion_last_week', 'amount_of_months_that_has_bought', 'timestamp_last_event', 
                                    'timestamp_last_checkout', 'timestamp_last_conversion', 'timestamp_last_viewed_product',
                                    'days_to_last_event', 'days_to_last_checkout', 'days_to_last_viewed_product', 'doy_last_event', 
                                    'dow_last_event', 'dom_last_event', 'woy_last_event', 'doy_last_checkout', 'dow_last_checkout', 
                                    'dom_last_checkout', 'woy_last_checkout', 'doy_last_conversion', 'dow_last_conversion', 
                                    'dom_last_conversion', 'doy_last_viewed_product', 'dow_last_viewed_product',
                                    'dom_last_viewed_product', 'woy_last_viewed_product'],
        
        'best_features_stepwise' : ['dow_last_conversion', 'timestamp_last_event', 'avg_events_per_session', 'woy_last_event', 'dow_last_checkout', 'doy_last_checkout', 'total_conversions_month_4', 'total_sessions', 'timestamp_last_checkout', 'has_checkout_month_1', 'has_checkout', 'total_checkouts', 'doy_last_conversion', 'has_conversion_last_week', 'has_checkout_month_5', 'total_session_checkouts']

}

In [ ]:
import nbimporter # pip install nbimporter
import pandas as pd
from parameter_tuning import get_hiper_params
import submission_framework as SF

df_users = pd.read_csv('data/user-features.csv',low_memory=False).set_index('person')
df_y = pd.read_csv('data/labels_training_set.csv').groupby('person').sum()

seed=42
hiper_params = get_hiper_params()

df_users.head()

---
---
---


Usando Random Forest, el algoritmo más estable de los definidos (XGBoost es poco estable) encontramos que combinación de features es la mas favorable (con la métrica Area Under Curve).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_name = 'random_forest'
params = hiper_params[model_name]
model = RandomForestClassifier(**params,random_state=seed)
random_forest = (model_name,model)

SF.full_framework_wrapper(df_users,df_y,random_forest)

In [ ]:
full_features = df_users.columns.tolist()
cantidad_features = len(full_features)

tried = {}

## Cumulative Importance

Se parte de una lista de todos los features ordeandos segun importancia, y se genera una lista de listas acumulativa de esto. Es decir de `[a,b,c]` se pasa a `[ [a], [a,b], [a,b,c] ]`

Esto se hace porque se esta buscando el 'codo': Los features que hacen que incremente el AUC.

In [ ]:
feature_importances = SF.get_feature_importances(df_users,df_y,random_forest)
features_ordenados = feature_importances.index.tolist()
lista_progresiva_de_cols = [features_ordenados[:i] for i in range(1,len(features_ordenados))]

model, max_auc = SF.full_framework_wrapper(df_users,df_y,random_forest)
best_features_progresivo = features_ordenados
features_con_saltos_progresivo = []

for i, cols in enumerate(lista_progresiva_de_cols):
    print(f'\n\nIteración {i+1} de {len(lista_progresiva_de_cols)}\n\n')
    if tuple(cols) not in tried:
        model, auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=cols,verbosity=1)
        tried[tuple(cols)] = auc
    else:
        auc = tried[tuple(cols)]
    if auc > max_auc + 0.0001:
        max_auc = auc
        best_features_progresivo = cols
        features_con_saltos_progresivo.append(cols[-1])

Model: random_forest - AUC: 0.7868 - AUCPR:0.1462 - Accuracy: 0.9496 


Iteración 1 de 336


Model: random_forest - AUC: 0.7659 - AUCPR:0.1427 - Accuracy: 0.9496 
['timestamp_last_checkout']


Iteración 2 de 336


Model: random_forest - AUC: 0.7684 - AUCPR:0.1435 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout']


Iteración 3 de 336


Model: random_forest - AUC: 0.7684 - AUCPR:0.1435 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout']


Iteración 4 de 336


Model: random_forest - AUC: 0.7683 - AUCPR:0.1435 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5']


Iteración 5 de 336


Model: random_forest - AUC: 0.7678 - AUCPR:0.1425 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5']


Iteración 6 de 336


Model: random_forest - AUC: 0.7683 - AUCPR:0.1435 - Accuracy: 0.9496 
['timestam

Model: random_forest - AUC: 0.8167 - AUCPR:0.1663 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5', 'total_session_checkouts_month_5', 'dom_last_checkout', 'days_between_last_event_and_checkout', 'days_to_last_checkout', 'dow_last_checkout', 'total_events_months_1_to_4', 'total_sessions_months_1_to_4', 'total_viewed_products', 'Samsung Galaxy Pocket 2 Duos', 'Samsung Galaxy On 7', 'Samsung Galaxy Note 8', 'Samsung Galaxy Note Edge', 'Samsung Galaxy S3 Mini', 'Samsung Galaxy Note 5', 'Samsung Galaxy Note 4', 'Samsung Galaxy Note 3 Neo Duos', 'Samsung Galaxy Note 3']


Iteración 23 de 336


Model: random_forest - AUC: 0.7963 - AUCPR:0.1567 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5', 'total_session_checkouts_month_5', 'dom_last_checkout', 'days_between_last_event_and_checkout', 'days_to_last_checkout', '

Model: random_forest - AUC: 0.7944 - AUCPR:0.1503 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5', 'total_session_checkouts_month_5', 'dom_last_checkout', 'days_between_last_event_and_checkout', 'days_to_last_checkout', 'dow_last_checkout', 'total_events_months_1_to_4', 'total_sessions_months_1_to_4', 'total_viewed_products', 'Samsung Galaxy Pocket 2 Duos', 'Samsung Galaxy On 7', 'Samsung Galaxy Note 8', 'Samsung Galaxy Note Edge', 'Samsung Galaxy S3 Mini', 'Samsung Galaxy Note 5', 'Samsung Galaxy Note 4', 'Samsung Galaxy Note 3 Neo Duos', 'Samsung Galaxy Note 3', 'Samsung Galaxy Note 2 N7100', 'Samsung Galaxy Mega Duos', 'Samsung Galaxy J7 Prime', 'Samsung Galaxy J7 PRO', 'Samsung Galaxy S3 Duos', 'Samsung Galaxy S4 Mini', 'Samsung Galaxy S3 Neo Duos i9300i', 'Samsung Galaxy S3 Slim Duos', 'Samsung Galaxy S3 i9300', 'Samsung Galaxy J7 2016 Metal', 'Samsung Galaxy S4 Mini Duos']


Iteración 34 d

Model: random_forest - AUC: 0.8101 - AUCPR:0.1643 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5', 'total_session_checkouts_month_5', 'dom_last_checkout', 'days_between_last_event_and_checkout', 'days_to_last_checkout', 'dow_last_checkout', 'total_events_months_1_to_4', 'total_sessions_months_1_to_4', 'total_viewed_products', 'Samsung Galaxy Pocket 2 Duos', 'Samsung Galaxy On 7', 'Samsung Galaxy Note 8', 'Samsung Galaxy Note Edge', 'Samsung Galaxy S3 Mini', 'Samsung Galaxy Note 5', 'Samsung Galaxy Note 4', 'Samsung Galaxy Note 3 Neo Duos', 'Samsung Galaxy Note 3', 'Samsung Galaxy Note 2 N7100', 'Samsung Galaxy Mega Duos', 'Samsung Galaxy J7 Prime', 'Samsung Galaxy J7 PRO', 'Samsung Galaxy S3 Duos', 'Samsung Galaxy S4 Mini', 'Samsung Galaxy S3 Neo Duos i9300i', 'Samsung Galaxy S3 Slim Duos', 'Samsung Galaxy S3 i9300', 'Samsung Galaxy J7 2016 Metal', 'Samsung Galaxy S4 Mini Duos', 'Samsung Galaxy 

Model: random_forest - AUC: 0.7944 - AUCPR:0.1500 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5', 'total_session_checkouts_month_5', 'dom_last_checkout', 'days_between_last_event_and_checkout', 'days_to_last_checkout', 'dow_last_checkout', 'total_events_months_1_to_4', 'total_sessions_months_1_to_4', 'total_viewed_products', 'Samsung Galaxy Pocket 2 Duos', 'Samsung Galaxy On 7', 'Samsung Galaxy Note 8', 'Samsung Galaxy Note Edge', 'Samsung Galaxy S3 Mini', 'Samsung Galaxy Note 5', 'Samsung Galaxy Note 4', 'Samsung Galaxy Note 3 Neo Duos', 'Samsung Galaxy Note 3', 'Samsung Galaxy Note 2 N7100', 'Samsung Galaxy Mega Duos', 'Samsung Galaxy J7 Prime', 'Samsung Galaxy J7 PRO', 'Samsung Galaxy S3 Duos', 'Samsung Galaxy S4 Mini', 'Samsung Galaxy S3 Neo Duos i9300i', 'Samsung Galaxy S3 Slim Duos', 'Samsung Galaxy S3 i9300', 'Samsung Galaxy J7 2016 Metal', 'Samsung Galaxy S4 Mini Duos', 'Samsung Galaxy 

...

Model: random_forest - AUC: 0.7804 - AUCPR:0.1504 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5', 'total_session_checkouts_month_5', 'dom_last_checkout', 'days_between_last_event_and_checkout', 'days_to_last_checkout', 'dow_last_checkout', 'total_events_months_1_to_4', 'total_sessions_months_1_to_4', 'total_viewed_products', 'Samsung Galaxy Pocket 2 Duos', 'Samsung Galaxy On 7', 'Samsung Galaxy Note 8', 'Samsung Galaxy Note Edge', 'Samsung Galaxy S3 Mini', 'Samsung Galaxy Note 5', 'Samsung Galaxy Note 4', 'Samsung Galaxy Note 3 Neo Duos', 'Samsung Galaxy Note 3', 'Samsung Galaxy Note 2 N7100', 'Samsung Galaxy Mega Duos', 'Samsung Galaxy J7 Prime', 'Samsung Galaxy J7 PRO', 'Samsung Galaxy S3 Duos', 'Samsung Galaxy S4 Mini', 'Samsung Galaxy S3 Neo Duos i9300i', 'Samsung Galaxy S3 Slim Duos', 'Samsung Galaxy S3 i9300', 'Samsung Galaxy J7 2016 Metal', 'Samsung Galaxy S4 Mini Duos', 'Samsung Galaxy 

Model: random_forest - AUC: 0.7887 - AUCPR:0.1517 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5', 'total_session_checkouts_month_5', 'dom_last_checkout', 'days_between_last_event_and_checkout', 'days_to_last_checkout', 'dow_last_checkout', 'total_events_months_1_to_4', 'total_sessions_months_1_to_4', 'total_viewed_products', 'Samsung Galaxy Pocket 2 Duos', 'Samsung Galaxy On 7', 'Samsung Galaxy Note 8', 'Samsung Galaxy Note Edge', 'Samsung Galaxy S3 Mini', 'Samsung Galaxy Note 5', 'Samsung Galaxy Note 4', 'Samsung Galaxy Note 3 Neo Duos', 'Samsung Galaxy Note 3', 'Samsung Galaxy Note 2 N7100', 'Samsung Galaxy Mega Duos', 'Samsung Galaxy J7 Prime', 'Samsung Galaxy J7 PRO', 'Samsung Galaxy S3 Duos', 'Samsung Galaxy S4 Mini', 'Samsung Galaxy S3 Neo Duos i9300i', 'Samsung Galaxy S3 Slim Duos', 'Samsung Galaxy S3 i9300', 'Samsung Galaxy J7 2016 Metal', 'Samsung Galaxy S4 Mini Duos', 'Samsung Galaxy 

Model: random_forest - AUC: 0.7965 - AUCPR:0.1549 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5', 'total_session_checkouts_month_5', 'dom_last_checkout', 'days_between_last_event_and_checkout', 'days_to_last_checkout', 'dow_last_checkout', 'total_events_months_1_to_4', 'total_sessions_months_1_to_4', 'total_viewed_products', 'Samsung Galaxy Pocket 2 Duos', 'Samsung Galaxy On 7', 'Samsung Galaxy Note 8', 'Samsung Galaxy Note Edge', 'Samsung Galaxy S3 Mini', 'Samsung Galaxy Note 5', 'Samsung Galaxy Note 4', 'Samsung Galaxy Note 3 Neo Duos', 'Samsung Galaxy Note 3', 'Samsung Galaxy Note 2 N7100', 'Samsung Galaxy Mega Duos', 'Samsung Galaxy J7 Prime', 'Samsung Galaxy J7 PRO', 'Samsung Galaxy S3 Duos', 'Samsung Galaxy S4 Mini', 'Samsung Galaxy S3 Neo Duos i9300i', 'Samsung Galaxy S3 Slim Duos', 'Samsung Galaxy S3 i9300', 'Samsung Galaxy J7 2016 Metal', 'Samsung Galaxy S4 Mini Duos', 'Samsung Galaxy 

Model: random_forest - AUC: 0.7946 - AUCPR:0.1557 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5', 'total_session_checkouts_month_5', 'dom_last_checkout', 'days_between_last_event_and_checkout', 'days_to_last_checkout', 'dow_last_checkout', 'total_events_months_1_to_4', 'total_sessions_months_1_to_4', 'total_viewed_products', 'Samsung Galaxy Pocket 2 Duos', 'Samsung Galaxy On 7', 'Samsung Galaxy Note 8', 'Samsung Galaxy Note Edge', 'Samsung Galaxy S3 Mini', 'Samsung Galaxy Note 5', 'Samsung Galaxy Note 4', 'Samsung Galaxy Note 3 Neo Duos', 'Samsung Galaxy Note 3', 'Samsung Galaxy Note 2 N7100', 'Samsung Galaxy Mega Duos', 'Samsung Galaxy J7 Prime', 'Samsung Galaxy J7 PRO', 'Samsung Galaxy S3 Duos', 'Samsung Galaxy S4 Mini', 'Samsung Galaxy S3 Neo Duos i9300i', 'Samsung Galaxy S3 Slim Duos', 'Samsung Galaxy S3 i9300', 'Samsung Galaxy J7 2016 Metal', 'Samsung Galaxy S4 Mini Duos', 'Samsung Galaxy 

Model: random_forest - AUC: 0.8047 - AUCPR:0.1509 - Accuracy: 0.9496 
['timestamp_last_checkout', 'doy_last_checkout', 'woy_last_checkout', 'has_checkout_month_5', 'total_checkouts_month_5', 'total_session_checkouts_month_5', 'dom_last_checkout', 'days_between_last_event_and_checkout', 'days_to_last_checkout', 'dow_last_checkout', 'total_events_months_1_to_4', 'total_sessions_months_1_to_4', 'total_viewed_products', 'Samsung Galaxy Pocket 2 Duos', 'Samsung Galaxy On 7', 'Samsung Galaxy Note 8', 'Samsung Galaxy Note Edge', 'Samsung Galaxy S3 Mini', 'Samsung Galaxy Note 5', 'Samsung Galaxy Note 4', 'Samsung Galaxy Note 3 Neo Duos', 'Samsung Galaxy Note 3', 'Samsung Galaxy Note 2 N7100', 'Samsung Galaxy Mega Duos', 'Samsung Galaxy J7 Prime', 'Samsung Galaxy J7 PRO', 'Samsung Galaxy S3 Duos', 'Samsung Galaxy S4 Mini', 'Samsung Galaxy S3 Neo Duos i9300i', 'Samsung Galaxy S3 Slim Duos', 'Samsung Galaxy S3 i9300', 'Samsung Galaxy J7 2016 Metal', 'Samsung Galaxy S4 Mini Duos', 'Samsung Galaxy 

In [ ]:
best_features_progresivo

##  Forward Selection

Se parte de una lista vacía y se van agregando todos los features uno por uno.

In [ ]:
def step_forward(features_usadas, features_sin_usar):
    if not features_sin_usar: 
        return SF.full_framework_wrapper(df_users,df_y,random_forest,columns=features_usadas,verbosity=1)[1], None
    max_auc = 0
    features_a_usar = None
    la_posta = None
    for f in features_sin_usar:
        features_a_probar = features_usadas + [f]
        if tuple(features_a_probar) not in tried: 
            model, auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=features_a_probar,verbosity=1)
            tried[tuple(features_a_probar)] = auc
        else: 
            auc = tried[tuple(features_a_probar)]
        if auc > max_auc + 0.0001:
            la_posta = f
            max_auc = auc
            
    return max_auc, la_posta

In [ ]:
CORTE = 10 # Si en 10 iteraciones no modifico su valor, se corta la ejecución.

contador = 0
best_features_forward = None
max_auc = 0
features_usadas = []
features_sin_usar = full_features[:]

for i in range(cantidad_features):
    print(f'\n\nIteración {i+1} de {cantidad_features}')
    print(f'Lo mejor al momento: {best_features_forward} (AUC: {max_auc:.4f}) \n\n')
    auc, feature_a_agregar = step_forward(features_usadas, features_sin_usar)
    features_usadas.append(feature_a_agregar)
    features_sin_usar.remove(feature_a_agregar)
    contador += 1
    if auc > max_auc + 0.0001:
        contador = 0
        best_features_forward = features_usadas[:]
        max_auc = auc   
    if contador == CORTE: break

In [ ]:
best_features_forward

##  Backward Elimination

Se parte de una lista con todos los features y se van sacando uno por uno, en busqueda de cual hace que incremente un AUC una vez que se lo remueva.

In [ ]:
def step_backward(features):
    if len(features) == 1: 
        print('atroden')
        return SF.full_framework_wrapper(df_users,df_y,random_forest,columns=features,verbosity=1)[1], None
    max_auc = 0
    features_a_usar = None
    la_posta = None
    for f in features:
        features_a_probar = features[:]
        features_a_probar.remove(f)
        if tuple(features_a_probar) not in tried:
            model, auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=features_a_probar,verbosity=1)
            tried[tuple(features_a_probar)] = auc
        else:
            auc = tried[tuple(features_a_probar)]
        if auc > max_auc + 0.0001:
            la_posta = f
            max_auc = auc
                
    return max_auc, la_posta

In [ ]:
contador = 0
features = full_features[:]
best_features_backward = features[:]
max_auc = SF.full_framework_wrapper(df_users,df_y,random_forest,columns=best_features_backward)[1]

for i in range(1,cantidad_features):
    print(f'\n\nIteración {i} de {cantidad_features-1}')
    print(f'Lo mejor al momento: {best_features_backward} (AUC: {max_auc:.4f}) \n\n')
    auc, feature_a_borrar = step_backward(features)
    features.remove(feature_a_borrar)
    contador += 1
    if auc > max_auc + 0.0001:
        contador = 0
        best_features_backward = features[:]
        max_auc = auc
    if contador == CORTE: break

Model: random_forest - AUC: 0.7868 - AUCPR:0.1462 - Accuracy: 0.9496 


Iteración 1 de 336
Lo mejor al momento: ['total_viewed_products', 'total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkout', 'total_session_conversion', 'total_events_ad_session', 'total_ad_sessions', 'avg_events_per_session', 'avg_events_per_ad_session', 'percentage_session_ad', 'percentage_session_conversion', 'has_checkout', 'has_conversion', 'total_viewed_products_month_1', 'total_checkouts_month_1', 'total_conversions_month_1', 'total_events_month_1', 'total_sessions_month_1', 'total_session_checkouts_month_1', 'total_session_conversions_month_1', 'total_events_ad_session_month_1', 'total_ad_sessions_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_viewed_products_month_2', 'total_checkouts_month_2', 'total_conversions_month_2', 'total_events_month_2', 'total_sessions_month_2', 'total_session_checkouts_month_2', 'total_session_conversions_month_2', 'to

Model: random_forest - AUC: 0.7811 - AUCPR:0.1624 - Accuracy: 0.9496 
['total_viewed_products', 'total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkout', 'total_session_conversion', 'total_events_ad_session', 'total_ad_sessions', 'avg_events_per_session', 'avg_events_per_ad_session', 'percentage_session_ad', 'percentage_session_conversion', 'has_checkout', 'has_conversion', 'total_viewed_products_month_1', 'total_checkouts_month_1', 'total_conversions_month_1', 'total_events_month_1', 'total_sessions_month_1', 'total_session_checkouts_month_1', 'total_session_conversions_month_1', 'total_events_ad_session_month_1', 'total_ad_sessions_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_viewed_products_month_2', 'total_checkouts_month_2', 'total_conversions_month_2', 'total_events_month_2', 'total_sessions_month_2', 'total_session_checkouts_month_2', 'total_session_conversions_month_2', 'total_events_ad_session_month_2', 'total_ad_

Model: random_forest - AUC: 0.7811 - AUCPR:0.1624 - Accuracy: 0.9496 
['total_viewed_products', 'total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkout', 'total_session_conversion', 'total_events_ad_session', 'total_ad_sessions', 'avg_events_per_session', 'avg_events_per_ad_session', 'percentage_session_ad', 'percentage_session_conversion', 'has_checkout', 'has_conversion', 'total_viewed_products_month_1', 'total_checkouts_month_1', 'total_conversions_month_1', 'total_events_month_1', 'total_sessions_month_1', 'total_session_checkouts_month_1', 'total_session_conversions_month_1', 'total_events_ad_session_month_1', 'total_ad_sessions_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_viewed_products_month_2', 'total_checkouts_month_2', 'total_conversions_month_2', 'total_events_month_2', 'total_sessions_month_2', 'total_session_checkouts_month_2', 'total_session_conversions_month_2', 'total_events_ad_session_month_2', 'total_ad_

Model: random_forest - AUC: 0.7811 - AUCPR:0.1624 - Accuracy: 0.9496 
['total_viewed_products', 'total_checkouts', 'total_conversions', 'total_events', 'total_sessions', 'total_session_checkout', 'total_session_conversion', 'total_events_ad_session', 'total_ad_sessions', 'avg_events_per_session', 'avg_events_per_ad_session', 'percentage_session_ad', 'percentage_session_conversion', 'has_checkout', 'has_conversion', 'total_viewed_products_month_1', 'total_checkouts_month_1', 'total_conversions_month_1', 'total_events_month_1', 'total_sessions_month_1', 'total_session_checkouts_month_1', 'total_session_conversions_month_1', 'total_events_ad_session_month_1', 'total_ad_sessions_month_1', 'has_checkout_month_1', 'has_conversion_month_1', 'total_viewed_products_month_2', 'total_checkouts_month_2', 'total_conversions_month_2', 'total_events_month_2', 'total_sessions_month_2', 'total_session_checkouts_month_2', 'total_session_conversions_month_2', 'total_events_ad_session_month_2', 'total_ad_

KeyboardInterrupt: 

In [ ]:
best_features_backward

##  Stepwise Regression

Combinación de Forward Selection y Backward Elimination

In [ ]:
contador = 0
best_features_stepwise = []
max_auc = 0
features_usadas = []
features_sin_usar = full_features[:]

for i in range(cantidad_features):
    print(f'\n\nIteración {i+1} de {cantidad_features}')
    print(f'Lo mejor al momento: {best_features_stepwise} (AUC: {max_auc:.4f}) \n\n')
    auc_loc, feature_a_agregar = step_forward(features_usadas, features_sin_usar)
    features_usadas.append(feature_a_agregar)
    features_sin_usar.remove(feature_a_agregar)
    features_to_stepwisear = features_usadas[:]
    iteraciones = len(features_to_stepwisear)
    
    print(f'\nStepWise: {features_to_stepwisear}\n')
    for i in range(1,iteraciones):
        auc, feature_a_borrar = step_backward(features_to_stepwisear)
        features_to_stepwisear.remove(feature_a_borrar)
        if auc > auc_loc:
            features_usadas = features_to_stepwisear[:]
            auc_loc = auc

    contador += 1
    if auc_loc > max_auc:
        contador = 0
        best_features_stepwise = features_usadas[:]
        max_auc = auc_loc   
        
    if contador == CORTE: break

In [ ]:
best_features_stepwise